## step 1

connect *source*.*ipynb* to Google Drive

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

print('project is connected to DRIVE')

## step 2
 change directory of workspace to **./drive/My Drive/erfan**

In [0]:
!ls
%cd ./drive/My Drive/erfan
!ls

## Import Libraries 
**keras** : using *keras* as a high level accessibility to *tensorflow*

**numpy** : using numpy in order to process images as matrices

**pandas** : using pandas for making *data_frame* from our *Corel* dataset

In [0]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
import pandas as pd
import numpy as np

print('Libraries Imported')

## step 3
read .CSV file in order to access *Corel* dataset and also converting it to a *string* format

[]((https://drive.google.com/open?id=132knE84o-7yzrvGPbjwXuftWmb6oKtf4) 

In [0]:
df = pd.read_csv("./Corel/newtrain2.csv", dtype = str)     
print('train.csv read!')
#print(df["class"])

## step 4

defining datas including *train_generator* (variable for our training data which is all of 10000 images from *Corel* dataset), *test_generator* (variable for our testing data) and *valid_generator* (variable for validation data)

In [0]:
datagen=ImageDataGenerator(rescale=1./255.)             #rescaling pixels (/255) for easier use and process
test_datagen=ImageDataGenerator(rescale=1./255.)        #rescaling pixels (/255) for easier use and process

train_generator = datagen.flow_from_dataframe(
    dataframe = df[:8000],
    directory = "./Corel",
    x_col = "image",
    y_col = "class",
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    color_mode = "rgb"
)

valid_generator = test_datagen.flow_from_dataframe(
    dataframe = df[8001:9000],
    directory= "./Corel",
    x_col = "image",
    y_col = "class",
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    color_mode = "rgb"
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = df[9001:10000],
    directory= "./Corel",
    x_col = "image",
    y_col = "class",
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    color_mode = "rgb"
)

## step 5

Designing a **Neural** **Network** model for training data in a format of **Sequential** model
input shape is (256, 256, 3) which : 

256*256 = the default format of our data_generators

3 = the color channels which are ''RGB'

and also the output of model is : (100) which are the 100 classes of our data

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(100, activation='sigmoid'))
model.add(Dense(80, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

print(model.input_shape)
print(model.output_shape)
print('model designed!')

## step 6
fitting the model which is traing our data

In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
"""
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)
"""

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=10
)

## step 7
predicting our data

First, we should reset *test_generator*

In [0]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [0]:
print('lenght of prediction : ',len(pred))
print('predictions : \n',pred)
print(len(test_generator.filenames))

In [0]:
pred_bool = (pred >0.007)
predictions = []
labels = train_generator.class_indices
print('labels : ', labels)
labels = dict((v,k) for k,v in labels.items())
print('lenght : ', len(labels))

for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
print('preditcionts : ', predictions, '\nit`s lenght :', len(predictions))
filenames=test_generator.filenames
print('preditcionts : ', predictions, '\nit`s lenght :', len(predictions))
print('filenames : ', filenames, '\nit`s lenght :', len(filenames))
results=pd.DataFrame({"image":filenames,"class":predictions})
print('done 1st!')
results.to_csv("results.csv",index=False)